In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

Lotka-Volterra model

In [ ]:
dynamical = dynamicals.LotkaVolterra()
config = core.Config()
config.create_time(0, 2, 100, 0, 2, 10, 0, 2, 10)
config.X_0 = np.array([5., 3.]) 
config.theta = np.array([2., 1., 1., 4.]) 
config.rho_2 = None
config.phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.sqrt([2.5, 0.02])),
    ('rbf', np.sqrt([2.5, 0.02]))
]
config.sigma_2 = np.array([0.1, 0.1]) 
config.delta = np.full(dynamical.num_x, True)
config.gamma = np.array([5e-3, 5e-3]) 

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = None
config.max_iter = 2000

config.plotting_enabled = False
config.plotting_freq = 50

config.debug = True

Lorenz 63 model

In [ ]:
dynamical = dynamicals.Lorenz63()
config = core.Config()
config.create_time(0, 10, 20, 0, 10, 2, 0, 10, 2)
config.X_0 = np.random.random(3) * 10.
config.theta = np.array([10., 28., 8. / 3])
config.rho_2 = None

config.phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.array([3.6, 0.15])),
    ('rbf', np.array([3.6, 0.15])),
    ('rbf', np.array([3.6, 0.15]))
]
config.sigma_2 = np.full(dynamical.num_x, 2.)
config.delta = np.full(dynamical.num_x, True)
config.gamma = np.full(dynamical.num_x, 1e-4) 

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = 5
config.max_iter = 2000

config.plotting_enabled = False
config.plotting_freq = 50

config.debug = True

Lorenz 96 model

In [ ]:
dynamical = dynamicals.Lorenz96()

config = core.Config()
config.create_time(0, 2, 40, 0, 2, 5, 0, 2, 5)

config.X_0 = np.random.random(dynamical.num_x) * 8.
config.theta = np.array([8.]) 
config.rho_2 = None
config.phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.array([4.5, 0.1]))
] * dynamical.num_x

config.sigma_2 = np.full(dynamical.num_x, 1.) 
config.delta = np.full(dynamical.num_x, True)
config.delta[np.random.permutation(dynamical.num_x)[:int(0.35 * dynamical.num_x)]] = False
config.gamma = np.full(dynamical.num_x, 5e-2) 
config.gamma[config.delta] = 1e-1

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = 10
config.max_iter = 1000

config.plotting_enabled = False
config.plotting_freq = 50

config.spl_X = dynamical.generate_sample_path(config.theta, config.rho_2, config.X_0, config.spl_tps)
config.obs_Y = utils.collect_observations(config.spl_X, config.obs_t_indices, config.sigma_2)

config.debug = True

In [ ]:
config.spl_X = dynamical.generate_sample_path(config.theta, config.rho_2, config.X_0, config.spl_tps)
config.obs_Y = utils.collect_observations(config.spl_X, config.obs_t_indices, config.sigma_2)

gp = core.GaussianProcessRegression(dynamical, config)
gp.run()

lpmf = core.LaplaceMeanFieldODE(dynamical, config, gp)
lpmf.init_optimization()

lpmf_sym = core.LaplaceMeanFieldSymbolic(dynamical, config, gp)
lpmf_sym.construct_X_objective_functions(False, False)
lpmf_sym.construct_theta_objective_functions(False, False)
lpmf_sym.init_optimization()

eta_X = lpmf.eta_X
eta_theta = lpmf.eta_theta
lpmf_sym.eta_theta[:] = lpmf.eta_theta.copy()

for it in range(5):
    print('Test case {}'.format(it + 1))
    for i in range(dynamical.num_x):
        # Simulating updating eta_X        
        eta_X[i] = np.random.random(config.est_tps.size) * 10

        lpmf.i = i
        x_objective, x_gradient = lpmf.x_objective_and_gradient(eta_X[i])
        x_hessian = lpmf.x_Hessian(eta_X[i])

        lpmf_sym.i = i
        x_objective_sym, x_gradient_sym = lpmf_sym.x_objective_and_gradient(eta_X[i])
        x_hessian_sym = lpmf_sym.x_Hessian(eta_X[i])

        print(
            'State', i + 1, '|',
            'Objective matched:', np.allclose(x_objective, x_objective_sym), '|',
            'Gradient matched:', np.allclose(x_gradient, x_gradient_sym), '|', 
            'Hessian matched:', np.allclose(x_hessian, x_hessian_sym)
        )        
    print()   
print('Test completed.')